<a href="https://colab.research.google.com/github/SanjanaA01/Projects/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip cache purge


Files removed: 29


In [70]:
!pip install numpy==1.26.4
!pip install jaxlib==0.1.75

# Install other dependencies
!pip install torch twilio yt-dlp pydub faster_whisper
!pip install --upgrade transformers accelerate
!pip install -q streamlit==1.29.0 pyngrok==7.0.0

# Verify installations
import transformers
#import jax
#import jaxlib


print(f"Transformers version: {transformers.__version__}")

ERROR: Ignored the following yanked versions: 0.4.32
ERROR: Could not find a version that satisfies the requirement jaxlib==0.1.75 (from versions: 0.4.17, 0.4.18, 0.4.19, 0.4.20, 0.4.21, 0.4.22, 0.4.23, 0.4.24, 0.4.25, 0.4.26, 0.4.27, 0.4.28, 0.4.29, 0.4.30, 0.4.31, 0.4.33, 0.4.34, 0.4.35, 0.4.36, 0.4.38, 0.5.0, 0.5.1, 0.5.3, 0.6.0, 0.6.1, 0.6.2, 0.7.0, 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.8.2)
ERROR: No matching distribution found for jaxlib==0.1.75
Transformers version: 4.57.3


In [71]:
from transformers import pipeline
import accelerate
import huggingface_hub

In [72]:
!ngrok config add-authtoken 37UamFkiM1VEpS80cmrZYU7e007_cGnGxwVCX1H9y52tNUTd
from pyngrok import ngrok

# Kill any previous instances of ngrok to avoid errors
ngrok.kill()
# Create a tunnel to the Streamlit app with the correct parameter
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Public URL: {public_url}")

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Public URL: NgrokTunnel: "https://raeann-postnasal-rocio.ngrok-free.dev" -> "http://localhost:8501"


In [78]:
!streamlit cache clear

Nothing to clear at /root/.streamlit/cache.


In [79]:
code = """
import streamlit as st
import os
from faster_whisper import WhisperModel
from transformers import pipeline
from pydub import AudioSegment
import yt_dlp

# ---------------- Page config ----------------
st.set_page_config(page_title="Fast Podcast Summarizer")
st.title("⚡ Fast Podcast Summarizer")

# ---------------- Session state ----------------
if "audio_path" not in st.session_state:
    st.session_state.audio_path = None
if "transcript" not in st.session_state:
    st.session_state.transcript = None
if "summary" not in st.session_state:
    st.session_state.summary = None

# ---------------- Models ----------------
@st.cache_resource
def load_whisper_model():
    return WhisperModel("tiny.en", device="cpu", compute_type="int8")

@st.cache_resource
def load_summarizer_model():
    return pipeline("summarization", model="t5-small")

# ---------------- UI ----------------
option = st.radio(
    "Choose input method",
    ["Upload Audio File", "YouTube Video Link"],
    key="input_method",
    horizontal=True,
)

# ---------------- Upload ----------------
if option == "Upload Audio File":
    uploaded = st.file_uploader("Upload audio", type=["mp3", "wav", "m4a"])
    if uploaded is not None:
        st.session_state.audio_path = uploaded.name
        with open(uploaded.name, "wb") as f:
            f.write(uploaded.getbuffer())
        st.success("✅ Audio uploaded successfully!")

# ---------------- YouTube ----------------
if option == "YouTube Video Link":
    url = st.text_input(
        "Paste YouTube URL",
        placeholder="https://www.youtube.com/watch?v=...",
    )

    if st.button("⬇️ Download Audio"):
        if not url:
            st.warning("⚠️ Please paste a YouTube URL first.")
        else:
            def download_hook(d):
                if d["status"] == "downloading":
                    downloaded = d.get("downloaded_bytes", 0)
                    total = d.get("total_bytes") or d.get("total_bytes_estimate")
                    if total and "yt_progress" in st.session_state:
                        progress = downloaded / total
                        st.session_state.yt_progress.progress(progress)

            st.session_state.yt_progress = st.progress(0)
            st.info("⏳ Downloading audio... This may take a while.")

            ydl_opts = {
                "format": "bestaudio/best",
                "outtmpl": "yt_audio.%(ext)s",
                "postprocessors": [
                    {
                        "key": "FFmpegExtractAudio",
                        "preferredcodec": "wav",
                    }
                ],
                "quiet": True,
                "nochechecertificate": True,
                "http_headers": {
                    "User-Agent": (
                        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/120.0.0.0 Safari/537.36"
                    ),
                    "Referer": "https://www.youtube.com/",
                },
                "progress_hooks": [download_hook],
            }

            try:
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([url])
                st.session_state.audio_path = "yt_audio.wav"
                st.session_state.yt_progress.progress(1.0)
                st.success("✅ Audio downloaded successfully!")
            except Exception as e:
                st.error(f"❌ Failed to download video: {e}")
                st.session_state.audio_path = None

st.divider()

# ---------------- Step 1: Transcribe ----------------
if st.session_state.audio_path:
    st.write("**Step 1: Transcription**")

    if st.session_state.transcript is None:
        if st.button("🚀 Transcribe Audio"):
            with st.spinner("🔄 Loading Whisper model..."):
                whisper = load_whisper_model()

            st.subheader("📝 Transcription Progress")
            progress_bar = st.progress(0)
            status_text = st.empty()

            try:
                # Load and limit audio
                status_text.text("Loading audio file...")
                audio = AudioSegment.from_file(st.session_state.audio_path)
                audio = audio[:5 * 60 * 1000]  # limit to 5 min
                audio.export("limited.wav", format="wav")

                # Transcribe
                status_text.text("Transcribing audio...")
                segments, _ = whisper.transcribe("limited.wav")
                transcript = ""
                segments = list(segments)
                total_segments = len(segments)

                if total_segments == 0:
                    st.error("❌ No audio segments found. The file might be empty or corrupted.")
                else:
                    for i, seg in enumerate(segments):
                        transcript += seg.text + " "
                        progress_bar.progress((i + 1) / total_segments)
                        status_text.text(f"Transcribing... {i + 1}/{total_segments} segments")

                    st.session_state.transcript = transcript
                    status_text.empty()
                    progress_bar.empty()
                    st.success("✅ Transcription completed!")
                    st.rerun()

            except Exception as e:
                st.error(f"❌ Transcription failed: {e}")
    else:
        st.success("✅ Transcription completed!")
        st.info(f"Transcript length: {len(st.session_state.transcript)} characters")

st.divider()

# ---------------- Step 2: Summarize ----------------
if st.session_state.transcript:
    st.write("**Step 2: Summarization**")

    if st.session_state.summary is None:
        if st.button("📝 Summarize Transcript"):
            with st.spinner("🔄 Loading summarization model..."):
                summarizer = load_summarizer_model()

            st.subheader("🤖 Summarization Progress")
            progress_bar = st.progress(0)
            status_text = st.empty()

            def chunk_text(text, max_chars=1200):
                for i in range(0, len(text), max_chars):
                    yield text[i : i + max_chars]

            try:
                chunks = list(chunk_text(st.session_state.transcript))
                summaries = []
                total_chunks = len(chunks)

                status_text.text(f"Summarizing {total_chunks} chunks...")

                for i, chunk in enumerate(chunks):
                    summaries.append(
                        summarizer(
                            chunk,
                            max_length=80,
                            min_length=30,
                            num_beams=1,
                            do_sample=False,
                        )[0]["summary_text"]
                    )
                    progress_bar.progress((i + 1) / total_chunks)
                    status_text.text(f"Summarizing... {i + 1}/{total_chunks} chunks")

                status_text.text("Creating final summary...")
                final_summary = summarizer(
                    " ".join(summaries),
                    max_length=120,
                    min_length=40,
                    num_beams=1,
                    do_sample=False,
                )[0]["summary_text"]

                st.session_state.summary = final_summary
                status_text.empty()
                progress_bar.empty()
                st.success("✅ Summarization completed!")
                st.rerun()

            except Exception as e:
                st.error(f"❌ Summarization failed: {e}")
    else:
        st.success("✅ Summarization completed!")

st.divider()

# ---------------- Display Results ----------------
if st.session_state.summary:
    st.subheader("📋 Summary")
    st.info(st.session_state.summary)

if st.session_state.transcript:
    with st.expander("📖 View Full Transcript"):
        st.text_area("Transcript", st.session_state.transcript, height=300)

# ---------------- Reset Button ----------------
if st.session_state.audio_path or st.session_state.transcript or st.session_state.summary:
    st.divider()
    if st.button("🔄 Start Over"):
        # Cleanup files
        for f in ["limited.wav", "yt_audio.wav"]:
            if os.path.exists(f):
                try:
                    os.remove(f)
                except:
                    pass

        # Reset session state
        st.session_state.audio_path = None
        st.session_state.transcript = None
        st.session_state.summary = None
        st.rerun()
"""
# Write the code to app.py
with open('app.py', 'w') as f:
    f.write(code)


In [80]:
!npm install -g localtunnel
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇Password/Enpoint IP for localtunnel is: 35.240.229.241
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://cold-oranges-rescue.loca.lt
^C
